In [1]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu7')

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:04:00.0)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 7: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import cPickle as pickle

In [3]:
def save_network(filename, param_values):
    with open(filename, 'wb') as f:
        pickle.dump(param_values, f, protocol=cPickle.HIGHEST_PROTOCOL)
        
def load_network(filename):
    with open(filename, 'rb') as f:
        param_values = pickle.load(f)
    return param_values

In [4]:
from models.inception_v3 import build_network

In [5]:
inception_data = load_network('models/inception_v3.pkl')
params = inception_data['param values']

In [6]:
net = build_network()
lasagne.layers.set_all_param_values(net.values(), params)

### Model

In [7]:
input_X = T.tensor4('X', dtype="float32")
target_y = T.vector('y', dtype="int64") 

In [8]:
hidden = lasagne.layers.get_output(net['pool3'], input_X, deterministic=True)

In [9]:
hidden_fun = theano.function([input_X], hidden, allow_input_downcast=True)

### Load data

In [11]:
from scipy.misc import imread, imresize

In [12]:
def preprocess(img):
    im = (img - 128) / 128.
    im = np.rollaxis(im, 3, start=1).astype(np.float32)
    return im

In [14]:
X_train = np.zeros((25000, 299, 299, 3), dtype=np.float32)
for i in xrange(12500):
    dog = imread('train/dog.%d.jpg' % i)
    cat = imread('train/cat.%d.jpg' % i)
    X_train[i] = imresize(dog, size=(299, 299))
    X_train[i + 12500] = imresize(cat, size=(299, 299))

In [15]:
X_test = np.zeros((1000, 299, 299, 3))
for i in xrange(1000):
    image = imread('test/%d.jpg' % i)
    X_test[i] = imresize(image, size=(299, 299))

In [16]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

### Forward

In [17]:
import tqdm

In [ ]:
X_train_inception = np.zeros((X_train.shape[0], 2048))
batchsize = 256
for start in tqdm.tqdm(xrange(0, X_train.shape[0], batchsize)):
    end = min(start + batchsize, X_train.shape[0])
    X_train_inception[start:end] = hidden_fun(X_train[start:end])

 63%|██████▎   | 62/98 [01:22<00:49,  1.36s/it]

In [ ]:
X_test_inception = np.zeros((X_test.shape[0], 2048))
batchsize = 256
for start in tqdm.tqdm(xrange(0, X_test.shape[0], batchsize)):
    end = min(start + batchsize, X_test.shape[0])
    X_test_inception[start:end] = hidden_fun(X_test[start:end])

In [19]:
np.savez('data_inception', X_test=X_test_inception, X_train=X_train_inception)